In [ ]:
!pip uninstall transformers -y
!pip install -U adapter-transformers
!pip install datasets
!pip install torch
!pip install datasets

Found existing installation: transformers 4.35.2
Uninstalling transformers-4.35.2:
  Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.6 MB/s eta 0:00:00


In [ ]:
labels = {0: 'operator_call',
1: 'emergency',
2: 'time_next_station',
3: 'people_next_station',
4: 'next_relax',
5: 'cur_time',
6: 'time_to_end_location',
7: 'trash'
}

In [ ]:
import datasets
import pandas as pd
from transformers import RobertaConfig, EvalPrediction, TrainingArguments,  RobertaTokenizer, TextClassificationPipeline, AutoModelForSequenceClassification, AdapterTrainer
import json
import torch
from typing import Tuple
from datasets import load_dataset
#import git
import os
import platform

In [ ]:
df_train = pd.read_csv('/content/MPIT_dataset_classifier.csv', delimiter=';')
for i in range(len(df_train)):
  if '"' in df_train['text'][i]:
    df_train['text'][i] = df_train['text'][i].replace('"', '')
  if '?' in df_train['text'][i]:
    df_train['text'][i] = df_train['text'][i].replace('?', '')



<ipython-input-5-7fc9d7e0fb8f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['text'][i] = df_train['text'][i].replace('"', '')
<ipython-input-5-7fc9d7e0fb8f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['text'][i] = df_train['text'][i].replace('?', '')


In [ ]:
#df_train = pd.read_csv('/content/MPIT_dataset_classifier.csv', delimiter=';')
#df_train = df_train[df_train['text'].str.contains('"')]
#df_train['text'] = df_train['text'].str.replace('"', '')


In [ ]:
df_train

,text,label
0,Можешь позвонить оператору пожалуйста,0
1,Соедини быстро с главным,0
2,Свяжи с оператором,0
3,"Подключи к оператору, пожалуйста.",0
4,"Позвони оператору и передай ему, что нужно сро...",0
...,...,...
536,Я спал в автобусе,7
537,Когда отдых,4
538,Когда я могу отдохнуть,4
539,через сколько отдых,4


In [ ]:
#from sklearn.model_selection import train_test_split
train_df = df_train.sample(frac=1, random_state=42)  # 42 - для воспроизводимости
#train_size = 0.8  # 80% для тренировочного датасета
#test_size = 0.2  # 20% для тестового датасета

#train_set, test_set = train_test_split(train_df, train_size=train_size, test_size=test_size, random_state=42)

#print("Размер тренировочного датасета:", len(train_set))
#print("Размер тестового датасета:", len(test_set))


In [ ]:
df_train.to_csv('dataset_MPIT.csv')

In [ ]:
df_train

,text,label
0,Можешь позвонить оператору пожалуйста,0
1,Соедини быстро с главным,0
2,Свяжи с оператором,0
3,"Подключи к оператору, пожалуйста.",0
4,"Позвони оператору и передай ему, что нужно сро...",0
...,...,...
536,Я спал в автобусе,7
537,Когда отдых,4
538,Когда я могу отдохнуть,4
539,через сколько отдых,4


In [ ]:
MODEL_NAME = 'ai-forever/ruRoberta-large'

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME)

vocab.json:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

In [ ]:
#labels = {v:k for k, v in labels.items()}

In [ ]:
dataset = load_dataset('csv', data_files={'train': ['/content/dataset_MPIT.csv']})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  print(batch.keys())
  return tokenizer(batch["text"], max_length=128, truncation=True, padding="max_length")

In [ ]:
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset = dataset.rename_column("label", "labels")
#dataset = dataset.rename_column("text", "text")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/541 [00:00<?, ? examples/s]

KeysView({'Unnamed: 0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 

In [ ]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    MODEL_NAME,
    num_labels=2,
)
model = RobertaModelWithHeads.from_pretrained(
    MODEL_NAME,
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/roberta/adapter_model.py:274: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/adapters/models/roberta/adapter_model.py:252: FutureWarning: This class has been renamed to `RobertaAdapterModel` in v3. Please use the new class instead as this class might be removed in a future version.
  warnings.warn(
Some weights of the model checkpoint at ai-forever/ruRoberta-large were not used when initializing RobertaModelWithHeads: ['lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model f

In [ ]:
# Add a new adapter
model.add_adapter("MPIT")
model.add_classification_head(

    "MPIT",
    num_labels=len(labels),
    id2label=labels
  )


In [ ]:
# Activate the adapter
model.train_adapter("MPIT")

In [ ]:
training_args = TrainingArguments(
    learning_rate=5e-5,
    num_train_epochs=2,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    logging_steps=20,
    output_dir="./training_output",
    overwrite_output_dir=False,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=True,

)

# def compute_accuracy(p: EvalPrediction):
#     preds = np.argmax(p.predictions, axis=1)
#     return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["train"],
     #compute_metrics=EvalPrediction,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaModelWithHeads.forward` and have been ignored: Unnamed: 0, text. If Unnamed: 0, text are not expected by `RobertaModelWithHeads.forward`,  you can safely ignore this message.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 541
  Num Epochs = 2
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 18
  Number of trainable parameters = 4229640


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=18, training_loss=0.080994360976749, metrics={'train_runtime': 50.357, 'train_samples_per_second': 21.487, 'train_steps_per_second': 0.357, 'total_flos': 255600459608064.0, 'train_loss': 0.080994360976749, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `RobertaModelWithHeads.forward` and have been ignored: Unnamed: 0, text. If Unnamed: 0, text are not expected by `RobertaModelWithHeads.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 541
  Batch size = 64


{'eval_loss': 0.03196307271718979,
 'eval_runtime': 13.8405,
 'eval_samples_per_second': 39.088,
 'eval_steps_per_second': 0.65,
 'epoch': 2.0}

In [ ]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

NameError: ignored

In [ ]:
classifier('Когда я мои пассажиры смогут поесть')

[[{'label': 'operator_call', 'score': 0.043677933514118195},
  {'label': 'emergency', 'score': 0.04773052781820297},
  {'label': 'time_next_station', 'score': 0.07314072549343109},
  {'label': 'people_next_station', 'score': 0.009442633017897606},
  {'label': 'next_relax', 'score': 0.32604900002479553},
  {'label': 'cur_time', 'score': 0.08457183092832565},
  {'label': 'time_to_end_location', 'score': 0.0446808785200119},
  {'label': 'trash', 'score': 0.3707064092159271}]]

In [ ]:
classifier.save_pretrained('/content/drive/MyDrive/MPIT_model')

Configuration saved in /content/drive/MyDrive/MPIT_model/config.json
Configuration saved in /content/drive/MyDrive/MPIT_model/generation_config.json
Model weights saved in /content/drive/MyDrive/MPIT_model/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/MPIT_model/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/MPIT_model/special_tokens_map.json


## Отсюда начинается классификация

In [ ]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    '/content/drive/MyDrive/MPIT_model/config.json',
    num_labels=2,
)
model = RobertaModelWithHeads.from_pretrained(
    '/content/drive/MyDrive/MPIT_model/pytorch_model.bin',
    config=config,
)

In [ ]:
model.set_active_adapters('MPIT')

In [ ]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('ai-forever/ruRoberta-large')


In [ ]:
training_args = TrainingArguments(
    learning_rate=5e-5,
    num_train_epochs=30,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    logging_steps=20,
    output_dir="./training_output",
    overwrite_output_dir=False,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=True,

)

In [ ]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

The model 'RobertaModelWithHeads' is not supported for . Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'EsmForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GPT2ForSequenceClassification', 'GPT2ForSequenceClassification', 'GPTNeoForSequenceClassification', 'GPTJForSequenceClassification', 'IBertForSequenceClassification', 'LayoutLMForSequenceClassification'